# Prise en main de Polars

Polars est un package Python permettant de manipuler les données tabulaires à partir de différents types de fichiers (CSV, Parquet, etc.). Il est une alternative directe et moderne à Pandas. Pour en savoir plus, la lecture de cet article du blog sur le site du SSPHub. > TODO lien

Pour bien débuter, on installe les packages nécessaires au fonctionnement de ce notebook et on importe toutes les fonctions à utiliser.


In [ ]:
!pip install polars pynsee[full] s3fs

In [2]:
import os
import polars as pl
import s3fs
from pynsee.download import download_file

# Lecture de données

Les exemples fournis dans ce notebook utiliseront les données de la BPE (à l'instar du [module de découverte du tidyverse dans utilitr](https://www.book.utilitr.org/03_fiches_thematiques/fiche_tidyverse)).

On exploite ici deux possibilités :
1. charger les données via le module Python `pynsee`
2. charger depuis le dossier `donnees-insee` du datalab TODO


## Via Pynsee

In [79]:
pandas_df_bpe = download_file("BPE_ENS") # pynsee renvoie un dataframe pandas
pandas_df_bpe["NB_EQUIP"] = [int(nb) for nb in pandas_df_bpe["NB_EQUIP"]]
df = pl.from_pandas(pandas_df_bpe)
df.head(5)

/opt/mamba/lib/python3.10/site-packages/pynsee/download/_download_store_file.py:68: UserWarning: File in insee.fr modified or corrupted during download
  warnings.warn("File in insee.fr modified or corrupted during download")


Previously saved data has been used:
/home/onyxia/.cache/pynsee/pynsee/fb1b844aa0ba3f4ac2490c59f518b09b
Set update=True to get the most up-to-date data


Extracting: 100%|██████████| 74.6M/74.6M [00:00<00:00, 212MB/s]


AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,i64
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""",1
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""",2
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""",1
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""",2
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""",2


## Via le stockage public du datalab

In [4]:
# WIP need to deal with types for parquet
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "donnees-insee/diffusion/BPE/2019"

with fs.open(f"{BUCKET}/BPE_ENS.csv") as bpe_csv:
    df_bpe = pl.read_csv(bpe_csv)
    print(df_bpe.head())
    #with fs.open(f"{BUCKET}/BPE_ENS.parquet", "w") as bpe_parquet:    
    df_bpe.write_parquet("bpe.parquet")

shape: (5, 1)
┌─────────────────────────────────────┐
│ REG";"DEP";"DEPCOM";"DCIRIS";"AN... │
│ ---                                 │
│ str                                 │
╞═════════════════════════════════════╡
│ 84;01;01001;01001;2019;A401;        │
│ 84;01;01001;01001;2019;A404;        │
│ 84;01;01001;01001;2019;A405;        │
│ 84;01;01001;01001;2019;A504;        │
│ 84;01;01001;01001;2019;A507;        │
└─────────────────────────────────────┘


In [80]:
df.write_parquet("bpe.parquet")
df_bpe = pl.read_parquet("bpe.parquet")

# Comment utiliser Polars ?

A l'instar d'autres outils modernes d'exploitation des données, Polars expose un modèle de traitement basé sur des fonctions de haut niveau, comme `select`, `filter` ou `groupby`, qui empruntent au langage SQL une logique expressive du "quoi ?" plutôt que du "comment ?".

Dans l'exemple qui suit, on commence par déclarer une exécution retardée (via `lazy()`) qui va permettre au moteur sous-jacent d'optimiser le traitement complet. Puis on exprime à l'aide des fonctions de haut niveau ce que l'on veut faire :
1. filtrer le jeu de données pour ne garder les lignes pour lesquelles la colonne `TYPEQU` vaut `B316` (les stations-services)
2. on regroupe au niveau département
3. on compte le nombre d'occurrences pour chaque département via `agg`
4. le dernier appel - `collect()` - indique que le traitement peut être lancé (et donc optimisé, parallelisé par Polars).

In [6]:
df_stations_service = df_bpe.lazy().filter( # 1.
    pl.col("TYPEQU") == "B316"
).groupby( # 2.
    "DEP"
).agg( # 3.
    pl.count().alias("NB_STATION_SERVICE")
).collect() # 4.

df_stations_service.head(5)

DEP,NB_STATION_SERVICE
str,u32
"""42""",97
"""73""",65
"""61""",38
"""35""",122
"""31""",155


## Lazy or not lazy ?

Les exemples qui suivent n'utilisent pas systématiquement la fonction `lazy()` avant les opérations sur le data frame pour montrer l'alternative qui s'offre à l'utilisateur de Polars. Cependant, l'usage de cette fonction est recommandée en général et en particulier pour des traiements sur des tables volumineuses : en effet, le mode _lazy_ permet au moteur sous-jacent d'optimiser les traitements et donc de gagner en rapidité (par exemple en parallélisant les calculs).

## Les expressions

...

## Sélection de données

Deux types de sélections sont possibles :
1. une sélection de variables (en colonne), avec `select`
2. une sélection d'observations (en ligne), avec `filter`

La combinaison des deux se faisant en chaînant l'appel à ces deux fonctions.

### Sélection de variables

On peut sélectionner des variables en utilisant leurs noms :

In [7]:
df_bpe.select(
    ["DEPCOM", "TYPEQU", "NB_EQUIP"]
).head(5)

DEPCOM,TYPEQU,NB_EQUIP
str,str,str
"""01001""","""A129""","""1"""
"""01001""","""A401""","""2"""
"""01001""","""A402""","""1"""
"""01001""","""A404""","""2"""
"""01001""","""A405""","""2"""


On peut également utiliser leurs positions :

In [8]:
df_bpe[:, 1:5].head(5)

AN,BV2012,DEP,DEPCOM
str,str,str,str
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""


Pour des usages plus avancé, on pourra s'appuyer sur des motifs de sélection des noms de colonnes mobilisant des expressions régulières (ici `^DEP.*$` signifiant "débute par DEP"):

In [9]:
df_bpe.select(
    pl.col("^DEP.*$")
).head(5)

DEP,DEPCOM
str,str
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""


La fonction `select` acceptant des `list` Python, on peut construire des sélecteurs assez puissants :

In [10]:
dep_cols = [cols for cols in df_bpe.columns if cols.startswith("DEP")] 

df_bpe.select(dep_cols).head(5)

DEP,DEPCOM
str,str
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""


### Sélection d'observations

La sélection d'observations (de lignes) se fera grâce à la fonction `filter`.

In [17]:
df_bpe.filter(
    (pl.col("DEPCOM") == "75101") &
    (pl.col("TYPEQU") == "B304")
).head(5)

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""001""","""2021""","""75056""","""75""","""75101""","""B""","""200054781""","""751010101""","""11""","""B3""","""B304""","""00851""","""2"""
"""001""","""2021""","""75056""","""75""","""75101""","""B""","""200054781""","""751010201""","""11""","""B3""","""B304""","""00851""","""17"""
"""001""","""2021""","""75056""","""75""","""75101""","""B""","""200054781""","""751010202""","""11""","""B3""","""B304""","""00851""","""3"""
"""001""","""2021""","""75056""","""75""","""75101""","""B""","""200054781""","""751010203""","""11""","""B3""","""B304""","""00851""","""6"""
"""001""","""2021""","""75056""","""75""","""75101""","""B""","""200054781""","""751010204""","""11""","""B3""","""B304""","""00851""","""7"""


De nombreux opérateurs sont disponibles pour faciliter cette sélection d'observations.

In [29]:
# Départements 75 ou 92
df_bpe.filter(
    pl.col("DEP").is_in(["75", "92"])
).head(5)

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""001""","""2021""","""75056""","""75""","""75101""","""A""","""200054781""","""751010201""","""11""","""A1""","""A101""","""00851""","""1"""
"""001""","""2021""","""75056""","""75""","""75101""","""A""","""200054781""","""751010301""","""11""","""A1""","""A101""","""00851""","""1"""
"""001""","""2021""","""75056""","""75""","""75101""","""A""","""200054781""","""751010402""","""11""","""A1""","""A101""","""00851""","""1"""
"""001""","""2021""","""75056""","""75""","""75101""","""A""","""200054781""","""751010102""","""11""","""A1""","""A105""","""00851""","""1"""
"""001""","""2021""","""75056""","""75""","""75101""","""A""","""200054781""","""751010402""","""11""","""A1""","""A120""","""00851""","""1"""


In [26]:
# Valeurs manquantes pour la variable EPCI
df_bpe.filter(
    pl.col("EPCI").is_null()
)

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str


## Renommage de variables

La fonction `rename` permet de lister les colonnes à renommer via un dictionnaire Python :

In [11]:
df_bpe.rename({
    "DEPCOM" : "code_commune"
}).head(5)

AAV2020,AN,BV2012,DEP,code_commune,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""","""2"""


_Fun fact_, la logique est l'inverse de celle de dplyr. :)

Comme vu plus haut, construire des expressions de renommage plus complexes pourra se faire en pur Python :

In [12]:
cols_minuscules = {cols:cols.lower() for cols in df_bpe.columns}

df_bpe.rename(cols_minuscules).head(5)

aav2020,an,bv2012,dep,depcom,dom,epci,dciris,reg,sdom,typequ,uu2020,nb_equip
str,str,str,str,str,str,str,str,str,str,str,str,str
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""","""2"""


## Tri d'une table

La fonction `sort` permet de trier la table sur une ou plusieurs variables.

In [40]:
df_bpe.lazy()\
    .sort("DEPCOM", "TYPEQU")\
    .head(5).collect()

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""","""2"""


L'ordre de tri - croissant par défaut - peut-être précisé pour chaque variable.

In [42]:
df_bpe.lazy()\
    .sort("DEPCOM", "TYPEQU", descending=[True, False])\
    .head(5).collect()

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""9F1""","""2021""","""CSZ""","""976""","""97617""","""A""","""200059871""","""97617_IND""","""06""","""A1""","""A128""","""9F304""","""1"""
"""9F1""","""2021""","""CSZ""","""976""","""97617""","""A""","""200059871""","""97617_IND""","""06""","""A1""","""A129""","""9F304""","""1"""
"""9F1""","""2021""","""CSZ""","""976""","""97617""","""A""","""200059871""","""97617_IND""","""06""","""A2""","""A203""","""9F304""","""1"""
"""9F1""","""2021""","""CSZ""","""976""","""97617""","""A""","""200059871""","""97617_IND""","""06""","""A2""","""A205""","""9F304""","""1"""
"""9F1""","""2021""","""CSZ""","""976""","""97617""","""A""","""200059871""","""97617_IND""","""06""","""A2""","""A206""","""9F304""","""1"""


## Création de nouvelles variables
La création de nouvelles variables se fera via `with_columns`. Dans l'exemple qui suit :
- on convertit en entier numérique la variable NB_EQUIP
- on calcule la somme cumulée (avec `cumsum`)
- on nomme la nouvelle colonne `NB_EQUIP_SUM`

In [61]:
df_bpe.lazy()\
    .with_columns(
        pl.col("NB_EQUIP").str.parse_int(radix=10).cumsum().alias("NB_EQUIP_SUM"),
        pl.when(pl.col("NB_EQUIP").str.parse_int(radix=10) > 3).then(True).otherwise(False).alias("NB_EQUIP_3PLUS")
    ).head(5).collect()

AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP,NB_EQUIP_SUM,NB_EQUIP_3PLUS
str,str,str,str,str,str,str,str,str,str,str,str,str,i32,bool
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1""",1,false
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2""",3,false
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""","""1""",4,false
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""","""2""",6,false
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""","""2""",8,false


## Production de synthèses et d'aggrégats
On peut produire des statistiques synthétiques sur notre jeu de données avec la fonction `select`.

In [81]:
df_bpe.lazy().select(
    pl.col("NB_EQUIP").sum().alias("NB_EQUIP_TOT")
).head(5).collect()

NB_EQUIP_TOT
i64
2399536


Les fonctions `describe` et `glimpse` proposent un aperçu synthétique de la table.

In [69]:
df_bpe.describe()

describe,AAV2020,AN,BV2012,DEP,DEPCOM,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914""","""1056914"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""001""","""2021""","""01004""","""01""","""01001""","""A""","""200000172""","""010010000""","""01""","""A1""","""A101""","""00151""","""1"""
"""max""","""SAR""","""2021""","""CSZ""","""976""","""97617""","""G""","""CSZ""","""97617_IND""","""94""","""G1""","""G104""","""CSZ""","""99"""
"""median""",null,null,null,null,null,null,null,null,null,null,null,null,null


In [70]:
df_bpe.glimpse()

Rows: 1056914
Columns: 13
$ AAV2020  <str> 524, 524, 524, 524, 524, 524, 524, 524, 524, 524
$ AN       <str> 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021
$ BV2012   <str> 01093, 01093, 01093, 01093, 01093, 01093, 01093, 01093, 01093, 01093
$ DEP      <str> 01, 01, 01, 01, 01, 01, 01, 01, 01, 01
$ DEPCOM   <str> 01001, 01001, 01001, 01001, 01001, 01001, 01001, 01001, 01001, 01001
$ DOM      <str> A, A, A, A, A, A, A, A, B, C
$ EPCI     <str> 200069193, 200069193, 200069193, 200069193, 200069193, 200069193, 200069193, 200069193, 200069193, 200069193
$ DCIRIS   <str> 010010000, 010010000, 010010000, 010010000, 010010000, 010010000, 010010000, 010010000, 010010000, 010010000
$ REG      <str> 84, 84, 84, 84, 84, 84, 84, 84, 84, 84
$ SDOM     <str> A1, A4, A4, A4, A4, A5, A5, A5, B2, C1
$ TYPEQU   <str> A129, A401, A402, A404, A405, A504, A505, A507, B203, C104
$ UU2020   <str> CSZ, CSZ, CSZ, CSZ, CSZ, CSZ, CSZ, CSZ, CSZ, CSZ
$ NB_EQUIP <str> 1, 2, 1, 2, 2, 1, 1, 2, 1, 1



Les fonctions `groupby` et `agg` sont utiles pour créer des aggrégats.

In [78]:
df_bpe.lazy().groupby("DEP").agg(
    (pl.col("TYPEQU") == "B203").sum().alias("NB_BOULANGERIES_TOT")
).sort("NB_BOULANGERIES_TOT", descending=True).head(5).collect()

DEP,NB_BOULANGERIES_TOT
str,u32
"""75""",731
"""59""",731
"""13""",576
"""69""",507
"""62""",440
